In [128]:
import requests
import os, shutil

# establishing session
s = requests.Session()

def load_user_data(page, session):
    url = 'http://cyberleninka.ru/article/c/matematika/%d' % (page)
    r = session.get(url)
    encoding = r.encoding if 'charset' in r.headers.get('content-type', '').lower() else None
    soup = BeautifulSoup(r.content, from_encoding=encoding)
    return soup

# clear the directory
for the_file in os.listdir('./html_pages/'):
    file_path = os.path.join('./html_pages/', the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)
        
# loading files
page = 1
while True:
    soup = load_user_data(page, s)
    article_list = soup.findAll('div', {'class': 'title'})
    if soup:
        for a in article_list:
            with open('./html_pages/page_%d.html' % (page), 'a') as output_file:
                output_file.write(a.text.encode('utf8') + '. \n')
    else:
        break
    page += 1

KeyboardInterrupt: 

In [129]:
with open('./text.html', 'w') as outfile:
    for fname in os.listdir('./html_pages/'):
        file_path = os.path.join('./html_pages/', fname)
        with open(file_path) as infile:
            outfile.write(infile.read())